# Preprocessing — Uruguay Census 2023 (PySpark)

This notebook contains the preprocessing stage of the Uruguay Census 2023 microdata.  
Its main goal is to clean, validate, and prepare the dataset for modeling by:

- handling special codes and outliers,  
- recoding categorical variables,  
- validating demographic fields such as age,  
- filtering invalid or unusable observations,  
- generating a clean and consistent version of the dataset.

This step follows the issues identified in the **01. EDA** notebook, and implements the required data cleaning tasks.


### **1. Environment Setup**


In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType

import pandas as pd

In [2]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "2g") \
    .appName("MyApp") \
    .getOrCreate()

### **2. Function definitions**


In [3]:
def summarize_category(df, column_name):
    """
    Computes frequency and percentage for a categorical column.
    
    Parameters:
        df : Spark DataFrame
        column_name : str
        
    Returns:
        Spark DataFrame with columns: column_name, count, percent
    """
    
    total = df.count()
    
    summary_df = (
        df.groupBy(column_name)
          .agg(
              F.count("*").alias("count"),
              (F.count("*") / total * 100).alias("percent")
          )
          .orderBy("count", ascending=False)
    )
    
    summary_df.show(truncate=False)
    return summary_df


### **3. Load Raw Dataset**

In [4]:
sdf = spark.read.csv("personas_ext_26_02.csv", header=True, inferSchema=True)

print("Rows:", sdf.count())
print("Columns:", len(sdf.columns))


Rows: 3499451
Columns: 102


In [5]:
sdf.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID_CENSO: double (nullable = true)
 |-- DIRECCION_ID: string (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- LOCALIDAD: integer (nullable = true)
 |-- VIVID: string (nullable = true)
 |-- HOGID: string (nullable = true)
 |-- PERID: string (nullable = true)
 |-- REGION_4: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- MUNICIPIO_PAIS: string (nullable = true)
 |-- TIPO_MUNICIPIO_PAIS: string (nullable = true)
 |-- FUENTE_EXT: integer (nullable = true)
 |-- SIT_CALLE: integer (nullable = true)
 |-- CUESTIONARIO_COMPLETO: integer (nullable = true)
 |-- CUESTIONARIO_BASICO: integer (nullable = true)
 |-- RRAA: integer (nullable = true)
 |-- UNIVERSO: integer (nullable = true)
 |-- VIVVO00: integer (nullable = true)
 |-- PERPH02: integer (nullable = true)
 |-- PERNA01: integer (nullable = true)
 |-- PERNA01_TRAMO: string (nullable = true)
 |-- PERPA01: integer (nullable = true)
 |-- PERPA02: integer (nulla

### **4. Standardizing Column Names**

For readability and consistency, we rename key variables as in the EDA stage.


In [6]:
sdf = (
    sdf.withColumnRenamed("ID_CENSO", "ID")
        .withColumnRenamed("PERNA01", "age")
        .withColumnRenamed("PERPH02", "sex")
        .withColumnRenamed("PERAL01", "works")
        .withColumnRenamed("PERED03", "education")
        .withColumnRenamed("MUNICIPIO_PAIS", "municipality")
        .withColumnRenamed("DEPARTAMENTO", "departament")
)

### **5. Cleaning `ID` variable**

In [7]:
sdf.select("ID").describe().show()

+-------+------------------+
|summary|                ID|
+-------+------------------+
|  count|           3499451|
|   mean|         1749726.0|
| stddev|1010204.6327705111|
|    min|               1.0|
|    max|         3499451.0|
+-------+------------------+



### **6. Cleaning `age` variable**

In [8]:
sdf.select("age").describe().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|           3499451|
|   mean|1007.2870181637062|
| stddev| 2099.288864007407|
|    min|                 0|
|    max|              5555|
+-------+------------------+



In [9]:
sdf.filter(F.col("age") >= 100) \
         .select("age") \
         .distinct() \
         .orderBy("age") \
         .show()

+----+
| age|
+----+
| 100|
| 101|
| 102|
| 103|
| 104|
| 105|
| 106|
| 107|
| 108|
| 109|
|5555|
+----+



In [10]:
sdf = sdf.filter((col("age").isNull()) | ((col("age") >= 0) & (col("age") <= 110)))

In [11]:
sdf.select("age").describe().show()


+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|           2884797|
|   mean|38.321099543572736|
| stddev|22.934068203811766|
|    min|                 0|
|    max|               109|
+-------+------------------+



### **7. Cleaning `sex` variable**

According to the official codebook:

| Code | Meaning |
|------|----------|
| **1** | Male (*Varón*) |
| **2** | Female (*Mujer*) |

These values were used in the preprocessing stage to recode `sex` into human-readable labels for analysis and modeling.


In [12]:
sdf.select("sex").distinct().count()

2

In [13]:
summarize_category(sdf, "sex")

+---+-------+-----------------+
|sex|count  |percent          |
+---+-------+-----------------+
|2  |1500009|51.99703826647074|
|1  |1384788|48.00296173352926|
+---+-------+-----------------+



DataFrame[sex: int, count: bigint, percent: double]

### **8. Cleaning `works` variable**

The original labor activity variable uses the following encoding:

| Code | Meaning          |
|------|------------------|
| **1** | Worked last week |
| **2** | Did not work     |

To improve interpretability in downstream models, this variable is recoded into a
binary indicator:

- **1** → the individual **worked**  
- **0** → the individual **did not work**

This transformation enhances model readability, simplifies feature usage, and
provides a clear and consistent representation of labor activity for
classification tasks.

In [14]:
sdf.select("works").distinct().count()

5

In [15]:
summarize_category(sdf, "works")

+-----+-------+-------------------+
|works|count  |percent            |
+-----+-------+-------------------+
|1    |1139942|39.515501437362836 |
|2    |1036888|35.94318768357011  |
|7777 |348142 |12.068162855133307 |
|8888 |347498 |12.04583892731447  |
|9898 |12327  |0.42730909661927685|
+-----+-------+-------------------+



DataFrame[works: int, count: bigint, percent: double]

In [16]:
sdf = sdf.withColumn(
    "works",
    F.when(F.col("works") == 1, 1)
     .when(F.col("works") == 2, 0)
     .otherwise(None)
)

In [17]:
summarize_category(sdf, "works")

+-----+-------+------------------+
|works|count  |percent           |
+-----+-------+------------------+
|1    |1139942|39.515501437362836|
|0    |1036888|35.94318768357011 |
|NULL |707967 |24.541310879067055|
+-----+-------+------------------+



DataFrame[works: int, count: bigint, percent: double]

### **9. Cleaning `education` variable**

#####  Official census coding and reordering

This variable corresponds to the highest level of education attained by the individual.  
According to the official coding used in the Uruguay Census 2023, the categories are:

| Code | Official Description |
|------|-----------------------|
| **1**  | Educación Inicial o Educación Preescolar |
| **2**  | Primaria común |
| **3**  | Primaria especial |
| **13** | Educación media básica — Ciclo Básico (Liceo o UTU) |
| **14** | Educación media superior — Bachillerato (Liceo o UTU) |
| **15** | Capacitaciones o cursos de UTU que *no* acreditan Ciclo Básico ni Bachillerato |
| **9**  | Magisterio o Profesorado |
| **10** | Terciario no universitario |
| **11** | Universidad o similar (Carrera de grado o Licenciatura) |
| **12** | Posgrado (diploma, maestría, doctorado) |

Although these values correctly represent the census categories, they are **not numerically ordered** from lowest to highest educational level.  
This can create difficulties when performing ordered analyses, generating visualizations, or training machine learning models that assume ordinal structure.

---

### Creating an ordered education variable

To fix this, an **ordinal version** of the education variable is created, assigning a numerical order strictly from the lowest to the highest level of formal education:

**Ordered classification (from lowest to highest):**

1. Inicial  
2. Primaria común  
3. Primaria especial  
4. UTU courses not accrediting Ciclo Básico  
5. Ciclo Básico (CB)  
6. Bachillerato  
7. Magisterio / Profesorado  
8. Terciario no universitario  
9. Universidad  
10. Posgrado  


In [18]:
sdf.select("education").distinct().count()

13

In [19]:
summarize_category(sdf, "education")

+---------+-------+-------------------+
|education|count  |percent            |
+---------+-------+-------------------+
|7777     |1840446|63.79811127091438  |
|8888     |350520 |12.150594998538892 |
|2        |205388 |7.119669078968122  |
|13       |127527 |4.420657675392757  |
|11       |112623 |3.9040182030139383 |
|14       |112078 |3.885126059129984  |
|1        |56640  |1.9633963845636278 |
|10       |20117  |0.6973454284651572 |
|9        |18333  |0.6355039886688735 |
|9898     |16588  |0.5750144637560286 |
|12       |10488  |0.36356111019250226|
|15       |7402   |0.2565865119798724 |
|3        |6647   |0.2304148264158622 |
+---------+-------+-------------------+



DataFrame[education: int, count: bigint, percent: double]

In [20]:
# sdf = sdf.withColumn("education", F.col("education").cast("int"))

In [21]:
sdf = sdf.withColumn(
    "education",
    F.when(F.col("education") == 1,  1) \
     .when(F.col("education") == 2,  2) \
     .when(F.col("education") == 3,  3) \
     .when(F.col("education") == 15, 4) \
     .when(F.col("education") == 13, 5) \
     .when(F.col("education") == 14, 6) \
     .when(F.col("education") == 9,  7) \
     .when(F.col("education") == 10, 8) \
     .when(F.col("education") == 11, 9) \
     .when(F.col("education") == 12, 10) \
     .otherwise(None)
)


In [22]:
summarize_category(sdf, "education")

+---------+-------+-------------------+
|education|count  |percent            |
+---------+-------+-------------------+
|NULL     |2207554|76.5237207332093   |
|2        |205388 |7.119669078968122  |
|5        |127527 |4.420657675392757  |
|9        |112623 |3.9040182030139383 |
|6        |112078 |3.885126059129984  |
|1        |56640  |1.9633963845636278 |
|8        |20117  |0.6973454284651572 |
|7        |18333  |0.6355039886688735 |
|10       |10488  |0.36356111019250226|
|4        |7402   |0.2565865119798724 |
|3        |6647   |0.2304148264158622 |
+---------+-------+-------------------+



DataFrame[education: int, count: bigint, percent: double]

### **10. Cleaning `departament` variable**


The variable **`department`** represents the official department code used in the 2023 Uruguay Census.  
This code ranges from **1 to 19**, covering all departments in the country.

The numbering follows this rule:

- **Code 1 corresponds to Montevideo**
- Codes **2 to 19 follow the alphabetical order** of the remaining departments

Below is the full mapping:

| Code | Department        |
|------|-------------------|
| 1    | Montevideo        |
| 2    | Artigas           |
| 3    | Canelones         |
| 4    | Cerro Largo       |
| 5    | Colonia           |
| 6    | Durazno           |
| 7    | Flores            |
| 8    | Florida           |
| 9    | Lavalleja         |
| 10   | Maldonado         |
| 11   | Paysandú          |
| 12   | Río Negro         |
| 13   | Rivera            |
| 14   | Rocha             |
| 15   | Salto             |
| 16   | San José          |
| 17   | Soriano           |
| 18   | Tacuarembó        |
| 19   | Treinta y Tres    |

In [23]:
sdf.select("departament").distinct().count()


19

In [24]:
summarize_category(sdf, "departament")

+-----------+-------+------------------+
|departament|count  |percent           |
+-----------+-------+------------------+
|1          |1302584|45.1534024751135  |
|3          |398873 |13.82672680261384 |
|10         |162636 |5.6376930508455185|
|15         |114084 |3.9546630144166124|
|5          |108630 |3.7656029176403054|
|16         |101976 |3.5349454398351075|
|11         |91720  |3.179426489974858 |
|13         |84775  |2.9386816472701542|
|18         |74796  |2.59276475953074  |
|4          |73392  |2.544095823726938 |
|17         |61990  |2.1488513749840976|
|2          |60060  |2.081948920495966 |
|12         |44781  |1.5523102665456183|
|14         |40322  |1.3977413315390996|
|6          |40279  |1.3962507587188977|
|9          |38645  |1.3396089915512253|
|8          |36471  |1.264248402920552 |
|19         |25890  |0.8974634956983107|
|7          |22893  |0.793574036578657 |
+-----------+-------+------------------+



DataFrame[departament: int, count: bigint, percent: double]

### **11. Cleaning `municipality` variable**


#### What is a *municipality* in Uruguay?
A **municipality** is a smaller administrative unit within a department.  
Municipalities group together towns, localities, or rural areas and are managed by local councils (*Municipios*).  
Not all areas in Uruguay are part of a municipality, but most populated zones are.


In [25]:
sdf.select("municipality").distinct().count()

67

In [26]:
summarize_category(sdf, "municipality")

+------------------+------+------------------+
|municipality      |count |percent           |
+------------------+------+------------------+
|Sin Municipio     |799026|27.697824144991834|
|Municipio A       |203020|7.037583580404444 |
|Municipio F       |168640|5.845818613926734 |
|Municipio D       |166657|5.777078941776493 |
|Municipio CH      |161715|5.605767060905845 |
|Municipio B       |154543|5.3571533802898434|
|Municipio G       |150336|5.2113198952993915|
|Municipio E       |149014|5.165493447199231 |
|Municipio C       |145852|5.055884348188105 |
|Maldonado         |99817 |3.460104818467296 |
|Ciudad de la Costa|73095 |2.5338004719222877|
|Las Piedras       |61478 |2.131103159078438 |
|Ciudad del Plata  |38726 |1.3424168147706754|
|Barros Blancos    |33807 |1.1719022170364155|
|Pando             |33574 |1.1638253922199726|
|San Carlos        |30280 |1.0496405812956684|
|Canelones         |28810 |0.9986837895352775|
|18 de Mayo        |24081 |0.8347554437972585|
|Carmelo     

DataFrame[municipality: string, count: bigint, percent: double]

##### Counting the Number of Municipalities per Department

In this section, we compute **how many municipalities exist within each department** in Uruguay based on the census dataset.

In [27]:
municipios_por_departamento = (
    sdf.groupBy("departament")
       .agg(F.countDistinct("municipality").alias("distinct_municipalities"))
       .orderBy("departament")
)

municipios_por_departamento.show(30, truncate=False)

+-----------+-----------------------+
|departament|distinct_municipalities|
+-----------+-----------------------+
|1          |9                      |
|2          |3                      |
|3          |31                     |
|4          |4                      |
|5          |13                     |
|6          |1                      |
|7          |1                      |
|8          |2                      |
|9          |2                      |
|10         |5                      |
|11         |3                      |
|12         |3                      |
|13         |1                      |
|14         |3                      |
|15         |2                      |
|16         |6                      |
|17         |3                      |
|18         |3                      |
|19         |2                      |
+-----------+-----------------------+



### **12. Final Clean Dataset**

In [28]:
sdf_clean = sdf.select(
    "ID",
    "age", 
    "sex",
    "works",
    "education",
    "municipality",
    "departament"
)

sdf_clean.show(10)

+----+---+---+-----+---------+-------------+-----------+
|  ID|age|sex|works|education| municipality|departament|
+----+---+---+-----+---------+-------------+-----------+
| 1.0| 74|  2| NULL|     NULL|  Municipio E|          1|
| 2.0| 37|  1| NULL|     NULL|  Municipio E|          1|
| 3.0| 39|  2| NULL|     NULL|  Municipio D|          1|
| 4.0| 14|  2| NULL|     NULL|  Municipio D|          1|
| 5.0| 19|  2| NULL|     NULL|  Municipio D|          1|
| 7.0| 61|  2| NULL|     NULL|  Las Piedras|          3|
| 8.0| 30|  2| NULL|     NULL|  Municipio A|          1|
| 9.0| 52|  2| NULL|     NULL|  Municipio A|          1|
|10.0| 52|  2| NULL|     NULL|Sin Municipio|         18|
|11.0| 67|  1| NULL|     NULL|     Progreso|          3|
+----+---+---+-----+---------+-------------+-----------+
only showing top 10 rows


### **13. Export Clean Dataset**

Export as Parquet

In [29]:
sdf_clean.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("dataset/personas_clean_csv")

In [30]:
sdf_clean.write.mode("overwrite").parquet("dataset/personas_clean.parquet")

# Then load the dataset as follows
# sdf_clean = spark.read.parquet("dataset/personas_clean.parquet")


In [32]:
spark.stop()

### **14. Next Steps**

This cleaned dataset will be used in **03. Modeling**, where feature engineering and machine-learning techniques will be applied.
